In [2]:
import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine

# Data Extraction

In [3]:
# Diccionario de pares
symbols = {
    "EURUSD": "EURUSD=X",
    "AUDUSD": "AUDUSD=X",
    "EURCHF": "EURCHF=X",
    "USDCAD": "USDCAD=X",
    "EURJPY": "EURJPY=X",
    "EURGBP": "EURGBP=X",
    "GBPJPY": "GBPJPY=X",
    "NZDUSD": "NZDUSD=X",
    "USDJPY": "USDJPY=X",
    "USDCHF": "USDCHF=X",
    "GBPUSD": "GBPUSD=X",
    "GBPCHF": "GBPCHF=X"
}

def descargar_divisas_merged(start_date, end_date, interval='15m'):
    """
    Descarga datos de varios pares desde Yahoo Finance y los combina
    en un único DataFrame, alineados por Datetime.
    
    Args:
        start_date (str): Fecha de inicio (YYYY-MM-DD)
        end_date (str): Fecha de fin (YYYY-MM-DD)
        interval (str): Intervalo de tiempo (default '15m')
    
    Returns:
        pd.DataFrame: DataFrame combinado con columnas separadas para cada par (sin Volume)
    """
    df_final = None

    for par, code in symbols.items():
        print(f"Descargando datos de {par}...")
        df = yf.download(
            tickers=code,
            start=start_date,
            end=end_date,
            interval=interval,
            progress=False
        )
        
        if not df.empty:
            df = df.reset_index()
            # Renombrar columnas para incluir el nombre del par (sin Volume)
            df_renamed = df.rename(columns={
                'Open': f'Open_{par}',
                'High': f'High_{par}',
                'Low': f'Low_{par}',
                'Close': f'Close_{par}'
            })
            # Mantener solo Datetime y las columnas renombradas que queremos
            columnas_necesarias = ['Datetime', f'Open_{par}', f'High_{par}', f'Low_{par}', f'Close_{par}']
            df_renamed = df_renamed[columnas_necesarias]
            
            if df_final is None:
                df_final = df_renamed
            else:
                # Unir por Datetime
                df_final = pd.merge(df_final, df_renamed, on='Datetime', how='outer')
        else:
            print(f"⚠️ No se obtuvieron datos para {par}")
    
    # Ordenar por Datetime
    if df_final is not None:
        df_final = df_final.sort_values(by='Datetime').reset_index(drop=True)
    else:
        print("❗ No se descargaron datos de ningún par.")
        df_final = pd.DataFrame()
    
    return df_final

In [20]:
datos = descargar_divisas_merged("2025-06-01", "2025-06-30")

Descargando datos de EURUSD...
Descargando datos de AUDUSD...
Descargando datos de EURCHF...
Descargando datos de USDCAD...
Descargando datos de EURJPY...
Descargando datos de EURGBP...
Descargando datos de GBPJPY...


C:\Users\haide\AppData\Local\Temp\ipykernel_18424\1013473878.py:59: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_final = pd.merge(df_final, df_renamed, on='Datetime', how='outer')
C:\Users\haide\AppData\Local\Temp\ipykernel_18424\1013473878.py:59: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_final = pd.merge(df_final, df_renamed, on='Datetime', how='outer')
C:\Users\haide\AppData\Local\Temp\ipykernel_18424\1013473878.py:59: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_final = pd.merge(df_final, df_renamed, on='Datetime', how='outer')
C:\Users\haide\AppData\Local\Temp\ipykernel_18424\1013473878.py:59: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_final = pd.merge(df_final, df_renamed, on='Datetime', how='outer')


Descargando datos de NZDUSD...
Descargando datos de USDJPY...
Descargando datos de USDCHF...
Descargando datos de GBPUSD...
Descargando datos de GBPCHF...


C:\Users\haide\AppData\Local\Temp\ipykernel_18424\1013473878.py:59: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_final = pd.merge(df_final, df_renamed, on='Datetime', how='outer')
C:\Users\haide\AppData\Local\Temp\ipykernel_18424\1013473878.py:59: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_final = pd.merge(df_final, df_renamed, on='Datetime', how='outer')
C:\Users\haide\AppData\Local\Temp\ipykernel_18424\1013473878.py:59: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  df_final = pd.merge(df_final, df_renamed, on='Datetime', how='outer')


# Transformación de datos

In [21]:
datos.columns = [' '.join(col).strip() if isinstance(col, tuple) else col for col in datos.columns]
datos.columns = [col[:-8] if len(col) > 8 else col for col in datos.columns]
datos.columns = [col.strip() for col in datos.columns]
datos.fillna(0, inplace=True)
datos

,Datetime,Open_EURUSD,High_EURUSD,Low_EURUSD,Close_EURUSD,Open_AUDUSD,High_AUDUSD,Low_AUDUSD,Close_AUDUSD,Open_EURCHF,...,Low_USDCHF,Close_USDCHF,Open_GBPUSD,High_GBPUSD,Low_GBPUSD,Close_GBPUSD,Open_GBPCHF,High_GBPCHF,Low_GBPCHF,Close_GBPCHF
0,2025-06-01 23:00:00+00:00,1.135847,1.136235,1.135718,1.136105,0.645078,0.645186,0.644828,0.645091,0.93346,...,0.82171,0.82195,1.346421,1.346928,1.346366,1.346801,1.10672,1.10700,1.10629,1.10691
1,2025-06-01 23:15:00+00:00,1.136105,1.136105,1.135976,1.136105,0.645078,0.645190,0.644953,0.645190,0.93346,...,0.82185,0.82193,1.346765,1.346820,1.346584,1.346765,1.10688,1.10708,1.10653,1.10684
2,2025-06-01 23:30:00+00:00,1.136235,1.136235,1.135976,1.136105,0.645186,0.645199,0.644995,0.645178,0.93354,...,0.82184,0.82202,1.346856,1.346928,1.346729,1.346928,1.10685,1.10736,1.10658,1.10733
3,2025-06-01 23:45:00+00:00,1.136235,1.136364,1.135460,1.135718,0.645178,0.645340,0.644579,0.644616,0.93366,...,0.82170,0.82220,1.347128,1.347400,1.346692,1.346892,1.10725,1.10748,1.10679,1.10743
4,2025-06-02 00:00:00+00:00,1.135718,1.137398,1.135718,1.137398,0.644608,0.645740,0.644608,0.645599,0.93348,...,0.82108,0.82118,1.347001,1.348909,1.347001,1.348909,1.10743,1.10770,1.10676,1.10759
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1891,2025-06-27 20:15:00+00:00,1.171783,1.171921,1.171783,1.171783,0.653048,0.653249,0.653048,0.653083,0.93639,...,0.79921,0.79937,1.371516,1.371836,1.371460,1.371460,1.09634,1.09666,1.09580,1.09630
1892,2025-06-27 20:30:00+00:00,1.171783,1.171921,1.171783,1.171921,0.653219,0.653471,0.653125,0.653441,0.93633,...,0.79905,0.79909,1.371535,1.371704,1.371385,1.371573,1.09631,1.09633,1.09568,1.09600
1893,2025-06-27 20:45:00+00:00,1.172608,1.172745,1.172058,1.172196,0.653125,0.653479,0.653083,0.653253,0.93624,...,0.79810,0.79836,1.371610,1.372363,1.370877,1.372363,1.09591,1.09601,1.09495,1.09592
1894,2025-06-27 21:00:00+00:00,1.171921,1.171921,1.171921,1.171921,0.653253,0.653253,0.653253,0.653253,0.93574,...,0.79836,0.79836,1.372288,1.372288,1.372288,1.372288,1.09546,1.09546,1.09546,1.09546


In [22]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1896 entries, 0 to 1895
Data columns (total 49 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   Datetime      1896 non-null   datetime64[ns, UTC]
 1   Open_EURUSD   1896 non-null   float64            
 2   High_EURUSD   1896 non-null   float64            
 3   Low_EURUSD    1896 non-null   float64            
 4   Close_EURUSD  1896 non-null   float64            
 5   Open_AUDUSD   1896 non-null   float64            
 6   High_AUDUSD   1896 non-null   float64            
 7   Low_AUDUSD    1896 non-null   float64            
 8   Close_AUDUSD  1896 non-null   float64            
 9   Open_EURCHF   1896 non-null   float64            
 10  High_EURCHF   1896 non-null   float64            
 11  Low_EURCHF    1896 non-null   float64            
 12  Close_EURCHF  1896 non-null   float64            
 13  Open_USDCAD   1896 non-null   float64            
 14  High_USD

# Carga datos a PostgreSQL

In [12]:
user = 'alumno'
password = '123456'
host = 'localhost'
port = '5432'
database = 'course-db'

engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

In [23]:
# Especifica el nombre de la tabla que deseas crear o actualizar
nombre_tabla = 'precios_forex'

# Enviar el DataFrame
datos.to_sql(nombre_tabla, engine, if_exists='replace', index=False)

562